# 2. Etapa de preprocesado de texto

In [33]:
from datasets import load_dataset
import pandas as pd
import nltk
import numpy as np
import matplotlib as plt
import multiprocessing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, TreebankWordTokenizer, RegexpTokenizer
from nltk import ngrams
from nltk.probability import FreqDist
from collections import Counter
from wordcloud import WordCloud
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from time import time
from stop_words import get_stop_words
import unicodedata
import re



In [35]:
# Importamos los datos
path = r'C:\Users\CARLES\1.CARLES\FORMACIONES\BOOTCAMP KEEPCODING\10. NLP'

df = pd.read_csv(f'{path}\df_nlp.csv')
reduced_df = pd.read_csv(f'{path}\\reduced_df_nlp.csv')
words = pd.read_csv(f'{path}\words_nlp.csv')


In [36]:
sw_list = get_stop_words('en')

In [37]:
def sentence_normalization(sentence):
    sentence = unicodedata.normalize('NFKD', sentence).lower().encode('ascii', errors='ignore').decode('utf-8')
    sentence = re.sub(' +', ' ', ' '.join([word if word.isalpha() else '' for word in sentence.split()])).strip()
    return sentence

In [38]:
def remove_stopwords(sentence, sw_list):
    sentence = ' '.join([word for word in sentence.split() if word not in sw_list])
    return sentence

In [39]:
def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(sentence)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    lemmatized_sentence = ' '.join(lemmatized_tokens)
    return lemmatized_sentence

In [40]:
# Funcion final

def process_reviews(reviews, sw_list):
    processed_sentences = []
    for sent in reduced_df['review']:
        if not sent != sent:  # check if sent is not nan
            sent = sentence_normalization(sent)
            sent = remove_stopwords(sent, sw_list)
            sent = lemmatize_sentence(sent)
            processed_sentences.append(sent)
        else:
            processed_sentences.append('None')
    return processed_sentences

In [41]:
processed_reviews = process_reviews(reduced_df['review'], sw_list)

In [42]:
print('Review original: {}'.format(reduced_df['review'].values[0]))
print('Review procesada: {}'.format(processed_reviews[0]))

Review original: This product does what I need it to do, I just wish it was odorless or had a soft coconut smell. Having my head smell like an orange coffee is offputting. (granted, I did know the smell was described but I was hoping it would be light)
Review procesada: product need just wish odorless soft coconut head smell like orange coffee know smell described hoping


In [43]:
reduced_df.loc[:, 'processedReview'] = processed_reviews
reduced_df

,review,sentiment,sentences,is_negative_sentiment,processedReview
0,"This product does what I need it to do, I just...",4.0,"['This product does what I need it to do, I ju...",0,product need just wish odorless soft coconut h...
1,Felt synthetic,1.0,['Felt synthetic'],1,felt synthetic
2,The polish was quiet thick and did not apply s...,4.0,['The polish was quiet thick and did not apply...,0,polish quiet thick apply let dry overnight add...
3,These were lightweight and soft but much too s...,3.0,['These were lightweight and soft but much too...,0,lightweight soft much small preferred two toge...
4,I was very disappointed when I got this facial...,3.0,['I was very disappointed when I got this faci...,0,disappointed got facial assumed like scrub use...
...,...,...,...,...,...
480797,I enjoy it for praying over people and anointi...,5.0,['I enjoy it for praying over people and anoin...,0,enjoy praying people anointing bed work
480798,It’s exactly as depicted!,5.0,['It’s exactly as depicted!'],0,exactly
480799,"Works well, so much better than putting foil o...",5.0,"['Works well, so much better than putting foil...",0,work much better putting foil love come dispen...
480800,I really enjoyed these masks. They felt calmin...,5.0,"['I really enjoyed these masks.', 'They felt c...",0,really enjoyed felt using skin looked clearer ...


In [44]:
reduced_df['processedReview'] = reduced_df['processedReview'].replace('', np.nan)
reduced_df = reduced_df.dropna(subset=['processedReview'])
reduced_df.head

<bound method NDFrame.head of                                                    review  sentiment  \
0       This product does what I need it to do, I just...        4.0   
1                                          Felt synthetic        1.0   
2       The polish was quiet thick and did not apply s...        4.0   
3       These were lightweight and soft but much too s...        3.0   
4       I was very disappointed when I got this facial...        3.0   
...                                                   ...        ...   
480797  I enjoy it for praying over people and anointi...        5.0   
480798                          It’s exactly as depicted!        5.0   
480799  Works well, so much better than putting foil o...        5.0   
480800  I really enjoyed these masks. They felt calmin...        5.0   
480801  A little goes a long way. It then washes off y...        5.0   

                                                sentences  \
0       ['This product does what I need it t

In [45]:
# Guardamos
reduced_df.to_csv(r'C:\Users\CARLES\1.CARLES\FORMACIONES\BOOTCAMP KEEPCODING\10. NLP\reduced_df_clean_nlp.csv', index = False)